#Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import os

from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
import torch

#Device checking

In [ ]:
import torch

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


#Defining class for custom class, loading pre-trained-GPT2 model, freezing some parameters and fine-tuning process.

**Checking the model architecture (Not mandatory)**

In [ ]:
# for name, param in model.named_parameters():
#   print(name)
#  # or
#     if 'specific_layer' in name:  # Replace 'specific_layer' with the name of the layer you want to fine-tune
#         param.requires_grad = True
#     else:
#         param.requires_grad = False

GPT2Block(
  (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
GPT2Block(
  (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
GPT2Block(
  (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  

#Fine-Tuning model

#**Load Trained model**

In [ ]:
# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained('/content/drive/My Drive/Subjects/NLP/Innovative/model/fine_tuned_gpt2_model')

#Method for generating text

In [ ]:
# model.to(device)
def generate_text(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    attention_mask = torch.ones_like(input_ids).to(device)

    output = model.generate(input_ids, attention_mask=attention_mask, num_beams=5,
                            max_length=max_length, num_return_sequences=1,no_repeat_ngram_size=2,early_stopping=True)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
    return '.'.join(generated_text.split('.')[:-1])+'.'

In [ ]:
# Define Dataset class for loading data from 'data.txt'
class MyDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = []
        for i in range(1,3):
          with open(os.path.join(file_path,f'feynman-lectures_part_{i}.txt'), 'r', encoding='utf-8') as file:
              temp=file.read()
              self.data+=temp.split('//par//')
              if '[topic]' in temp:
                self.data+=temp.split('[topic]')
              if '//topic//' in temp:
                self.data+=temp.split('//topic//')
              self.data+=temp.split('//chap//')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokenized_data = self.tokenizer.encode_plus(
            self.data[idx],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return tokenized_data.input_ids, tokenized_data.attention_mask

# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token

# Freeze parameters of the model except for the last layer
for param in model.parameters():
    param.requires_grad = False
for param in model.transformer.h[-2:].parameters():
    param.requires_grad = True



In [ ]:
# Prepare dataset and dataloader
dataset = MyDataset('/content/drive/My Drive/Subjects/NLP/Innovative/Version_2/Data2', tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
len(dataset.data)

72

#**fine-tuning**

In [ ]:

# Define training configuration
learning_rate = 5e-5
num_epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to device
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Fine-tune the model
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, attention_masks in dataloader:
        inputs = inputs.to(device)
        attention_masks = attention_masks.to(device)

        # Forward pass
        outputs = model(input_ids=inputs, attention_mask=attention_masks, labels=inputs)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}')

# Save the fine-tuned model
model.save_pretrained('fine_tuned_gpt2_model')

Epoch [1/30], Loss: 1.7698
Epoch [2/30], Loss: 1.6723
Epoch [3/30], Loss: 1.6091
Epoch [4/30], Loss: 1.5521
Epoch [5/30], Loss: 1.5101
Epoch [6/30], Loss: 1.4588
Epoch [7/30], Loss: 1.4195
Epoch [8/30], Loss: 1.3838
Epoch [9/30], Loss: 1.3459
Epoch [10/30], Loss: 1.3168
Epoch [11/30], Loss: 1.2766
Epoch [12/30], Loss: 1.2399
Epoch [13/30], Loss: 1.2053
Epoch [14/30], Loss: 1.1818
Epoch [15/30], Loss: 1.1373
Epoch [16/30], Loss: 1.1056
Epoch [17/30], Loss: 1.0865
Epoch [18/30], Loss: 1.0385
Epoch [19/30], Loss: 1.0114
Epoch [20/30], Loss: 0.9780
Epoch [21/30], Loss: 0.9485
Epoch [22/30], Loss: 0.9190
Epoch [23/30], Loss: 0.8910
Epoch [24/30], Loss: 0.8624
Epoch [25/30], Loss: 0.8348
Epoch [26/30], Loss: 0.8003
Epoch [27/30], Loss: 0.7794
Epoch [28/30], Loss: 0.7581
Epoch [29/30], Loss: 0.7361
Epoch [30/30], Loss: 0.7188


In [ ]:
model.save_pretrained('/content/drive/My Drive/Subjects/NLP/Innovative/Version_2/trained_model')

#**Generate text**

#(1) by version-1

In [ ]:
generate_text('What is the motion of molecules of the water in ice? ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'What is the motion of molecules of the water in ice? \n\nWater molecules move in a very simple way—by passing through a thin membrane of water molecules called a pore. Water molecules have a diameter of about 1/8 of an inch (1/16 of a millimeter). When they are moving through water, they bump up and down a membrane which acts as a sort of transducer.'

#(2) by version-2

In [ ]:
generate_text('What is the motion of molecules of the water in ice? ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'What is the motion of molecules of the water in ice? \n\nThe atoms of water droplets have a few special properties. First of all, they do not attract each other, but they attract one another. They attract so much so fast that there is virtually no chance for the atoms to get so close together, and so there ice is formed. The water is so big that it is able to hold the many water molecules in its grooves all by itself.'

#Additional examples

#(1) By verion-1

In [ ]:
generate_text('What is the theory of relativity? Explain both genera and special theory of relativity')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'What is the theory of relativity? Explain both genera and special theory of relativity.\nIn the early days of space-time, there was no such thing as general relativity—there was only general-relativistic mechanics. General relativity says that all things in the universe are made of the same three-dimensional things, and that there is only one known form of such things.'

# By version-2

In [ ]:
generate_text('What is the theory of relativity? Explain both genera and special theory of relativity')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'What is the theory of relativity? Explain both genera and special theory of relativity. I will try to explain both of them in less than 5 minutes.\nConsider the following:\nImagine a pendulum (in this example, made of course, not a real thing) with a length of one foot. Now, imagine that the mass is proportional to the distance from the center. Let us call this distance the real mass.'

#(1) By version-1

In [ ]:
generate_text('Explain me the concept of atomic structure')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Explain me the concept of atomic structure.\nIn the early days of physics, there was a great deal of interest in the idea of what we now call atomic mechanics. In the late 19th century, the work of Hermann Heisenberg and other physicists, first demonstrated the laws of heat, and then more recently, of the conservation of energy, were put to the test in a series of high-energy nuclear devices (called fissile material physics devices).'

#(2) By version-2

In [ ]:
generate_text('Explain me the concept of atomic structure')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Explain me the concept of atomic structure. It is not a new one; it was described in the first periodic chart of Mendeleev. We have described it in its simplest form: if we have not defined it yet, we shall do so in due time.\nWe do not know what it is about, but we do know that is all right. The properties of matter are, as has been said, not reducible to the vibrations of vibrations.'

#(1)

In [ ]:
generate_text('Explain electrons in atom')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Explain electrons in atom- and molecule-level chemistry\nLet us now consider the chemical processes that take place in an object when it is in motion. In the process of chemical reactions, the atoms are jiggled around in a myriad of possible combinations until they find their final form.\nIn a chemical reaction, a substance is combined with a large number of partners, and the product is then broken down by the various partners into smaller and smaller molecules.'

#(2)

In [ ]:
generate_text('Explain electrons in atom')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Explain electrons in atom theory\nLet us suppose that the electron can be thought of as a particle that can move around in a certain way. So, in general, if we have an atom with some distance d, we say that d is the distance from the center of the atom, i.e., the point of inertia at which all the atoms are jiggling around.'

#Other examples

In [ ]:
generate_text('how electrons revolves around proton ? ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'how electrons revolves around proton? ices around protons. The electrons are in perpetual motion to each other, so they revolve around the nucleus. So in the year of the arrow, the electron has to be in motion around in some period of time to keep it from being out of whirlpool forever and forever. It keeps on going in a certain direction. That is how long it takes for it to get from the center to the edge.'

In [ ]:
generate_text('kinetic energy of particle')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'kinetic energy of particle accelerators\nIn this chapter we\nshall consider the most efficient means of accelerating particles to the speed of light, where light is a description of particles.\nFirst, a few things need to be noted. In the early days of quantum mechanics, the way that light propagates was not well understood. To illustrate the point, consider an object moving down an inclined trough.'

In [ ]:
generate_text('wave functions of electron according to quantum mechanics ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'wave functions of electron according to quantum mechanics \nQuantum mechanics is the description of the behavior of matter, as well as of light. In the years before 1920, the electron, and its spin, were thought to be made up of particles which move around in a manner analogous to the way light does. But in 1926, Born and Dirac, both physicists, first proposed that the spin of an atom, i.e., its charge, was not a particle at all.'

In [ ]:
s=generate_text('wave functions of electron according to quantum mechanics')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
s

'wave functions of electron according to quantum mechanics.\n\n[topic] Gravitational-wave function of quantum-mechanical oscillations\nConservation of energy can be thought of two forms:\n1. The form in which light is scattered by spacetime wave amplitudes in all directions, and which has the property that it does not interact with the light in any definite way. It obeys the Schrödinger equation: Eq. (4.'